In [7]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import pandas as pd
import cv2
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [2]:
data_labels = pd.read_csv("/Users/jarvis/pymycod/tensorflow_AI/dog-breed-identification-2/labels.csv")
data_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
data_labels.describe()

,id,breed
count,10222,10222
unique,10222,120
top,000bec180eb18c7604dcecc8fe0dba07,scottish_deerhound
freq,1,126


In [4]:
total_breeds = data_labels["breed"].unique()
len(total_breeds)

120

In [13]:
train_file = '/Users/jarvis/pymycod/tensorflow_AI/dog-breed-identification-2/train/'
test_file = '/Users/jarvis/pymycod/tensorflow_AI/dog-breed-identification-2/train/'

In [5]:
breed_dict = list(data_labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:60*2+1:2]
#change the dataset to have only those 60 unique breed records
data_labels = data_labels.query('breed in @new_list')
data_labels['img_file'] = data_labels['id'].apply(lambda x: x + ".jpg")

In [55]:
data_labels.head()

,id,breed,img_file
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,0021f9ceb3235effd7fcde7f7538ed62.jpg


In [62]:
breed_dict = list(data_labels['breed'].value_counts().keys()) 
# new_list = sorted(breed_dict,reverse=True)
new_list = sorted(breed_dict,reverse=True)
new_list

['yorkshire_terrier',
 'whippet',
 'welsh_springer_spaniel',
 'walker_hound',
 'toy_terrier',
 'tibetan_terrier',
 'sussex_spaniel',
 'standard_poodle',
 'soft-coated_wheaten_terrier',
 'siberian_husky',
 'shetland_sheepdog',
 'scottish_deerhound',
 'schipperke',
 'saluki',
 'rottweiler',
 'redbone',
 'pomeranian',
 'pekinese',
 'otterhound',
 'norwich_terrier',
 'norfolk_terrier',
 'miniature_schnauzer',
 'miniature_pinscher',
 'maltese_dog',
 'malamute',
 'leonberg',
 'labrador_retriever',
 'komondor',
 'kelpie',
 'japanese_spaniel',
 'irish_wolfhound',
 'irish_terrier',
 'ibizan_hound',
 'greater_swiss_mountain_dog',
 'great_dane',
 'golden_retriever',
 'german_short-haired_pointer',
 'french_bulldog',
 'eskimo_dog',
 'english_springer',
 'english_foxhound',
 'dingo',
 'dandie_dinmont',
 'collie',
 'clumber',
 'chihuahua',
 'cardigan',
 'bull_mastiff',
 'briard',
 'boxer',
 'boston_bull',
 'border_terrier',
 'bluetick',
 'blenheim_spaniel',
 'bernese_mountain_dog',
 'beagle',
 'base

In [59]:
encoder = LabelEncoder()
im_size = 224

In [14]:

train_x = np.zeros((len(data_labels), im_size, im_size, 3), dtype='float32')
for i, img_id in enumerate(data_labels['img_file']):
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  train_x[i] = img_array

In [15]:
train_y = encoder.fit_transform(data_labels["breed"].values)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [17]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=64)
 
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_test, 
                                     y_test, 
                                     batch_size=64)

In [18]:
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-10-05 15:20:32.841461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-05 15:20:32.843018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


94668760/94668760 [==============================] - 15s 0us/step


In [19]:
predictions = Dense(60, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=predictions)


In [21]:
from keras.callbacks import ModelCheckpoint
model_path = "/Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_dog_breed.h5"
checkpoint = ModelCheckpoint(model_path,monitor="val_accuracy",verbose=1,save_best_only=True,mode=max)
callbacks_list=[checkpoint]

In [22]:

epochs = 20
learning_rate = 1e-3
 

optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
 
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // 64,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_test.shape[0] // 64,callbacks=callbacks_list)
 


Epoch 1/20


2023-10-05 15:27:29.056267: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-05 15:27:30.899737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


64/64 [==============================] - ETA: 0s - loss: 2.2133 - accuracy: 0.4308

2023-10-05 15:28:34.051390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_accuracy improved from -inf to 0.76074, saving model to /Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_dog_breed.h5
64/64 [==============================] - 82s 1s/step - loss: 2.2133 - accuracy: 0.4308 - val_loss: 0.7832 - val_accuracy: 0.7607
Epoch 2/20
64/64 [==============================] - ETA: 0s - loss: 1.2762 - accuracy: 0.6241
Epoch 2: val_accuracy improved from 0.76074 to 0.78320, saving model to /Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_dog_breed.h5
64/64 [==============================] - 52s 810ms/step - loss: 1.2762 - accuracy: 0.6241 - val_loss: 0.6936 - val_accuracy: 0.7832
Epoch 3/20
64/64 [==============================] - ETA: 0s - loss: 1.0838 - accuracy: 0.6752
Epoch 3: val_accuracy improved from 0.78320 to 0.79688, saving model to /Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_dog_breed.h5
64/64 [==============================] - 55s 849ms/step - loss: 1.0838 - accuracy: 0.6752 - val_loss: 0.6672 - val_acc

KeyboardInterrupt: 

In [77]:
model = load_model("/Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_dog_breed.h5")


In [103]:
pred_img_path = '/Users/jarvis/Downloads/images-95.jpeg'
pred_img_array = cv2.resize(cv2.imread(pred_img_path,cv2.IMREAD_COLOR),((im_size,im_size)))
pred_img_array = preprocess_input(np.expand_dims(np.array(pred_img_array[...,::-1].astype(np.float32)).copy(), axis=0))
pred_val = model.predict(np.array(pred_img_array,dtype="float32"))

1/1 [==============================] - 0s 93ms/step


In [104]:
pred_breed = sorted(new_list)[np.argmax(pred_val)]

In [106]:
print("Predicted Breed for this Dog is :",pred_breed)

Predicted Breed for this Dog is : miniature_pinscher
